In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

# RunnablePassthrough
- It does not do anything to the input data.
- Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("SiZ")

'SiZ'

# RunnableLambda
- To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
- Let's define a very simple function to create Russian lastnames:

In [5]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [ ]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)
# chain = RunnableLambda(russian_lastname)

In [17]:
chain.invoke("SiZ")

'SiZovich'

# RunnableParallel
- We will use RunnableParallel() for running tasks in parallel.
- This is probably the most important and most useful Runnable from LangChain.
- In the following chain, RunnableParallel is going to run these two tasks in parallel:
    + operation_a will use RunnablePassthrough.
    + operation_b will use RunnableLambda with the russian_lastname function.

In [19]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [20]:
chain.invoke("SiZ")

{'operation_a': 'SiZ', 'operation_b': 'SiZovich'}

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [22]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [23]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [24]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [ ]:
chain.invoke({
    "name1": "Jordam", # Name1 not given
    "name": "Abram"
})

'A curious fact about Roman Abramovich is that before he became a billionaire businessman and owner of Chelsea Football Club, he started out with very modest beginnings in post-Soviet Russia. After the collapse of the Soviet Union, Abramovich capitalized on the new opportunities during Russia’s privatization era, acquiring stakes in oil companies. One notable—and somewhat ironic—story is that he reportedly bought the oil company Sibneft at a fraction of its value during a controversial privatization auction, which later made him one of the richest men in Russia. This rapid rise from a relatively modest background to immense wealth is often cited as emblematic of the chaotic economic changes in Russia during the 1990s.'

Advanced RunnableParallel

In [28]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["SiZ primarily focused contents on game, highlights, strategies, and tips for players."], 
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4.1-mini")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()}) # passing both
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is SiZ?")

'SiZ is a content creator who primarily focuses on game-related content, including highlights, strategies, and tips for players.'

In [31]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-4.1-mini")

vectorstore = FAISS.from_texts(
    ["SiZ primarily focused contents on game, highlights, strategies, and tips for players."], 
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is SiZ?", "language": "Pirate English"})

"Arrr, SiZ be a treasure trove o' game contents, sharin' highlights, strategies, and tips fer all ye players sailin' the digital seas!"